In [25]:
import pandas as pd
import os
from pathlib import Path
import glob

In [26]:
# Rutas
ROOT_PATHS = [
    Path("0 REPORTES DIARIOS 2024"),
    Path("1 REPORTES DIARIOS 2025")
]
OUTPUT_PATH = Path("procesados")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [27]:
# Mapeo de nombres de los archivos
FILES_CONFIG = {
    "Generator speed": "GEN",
    "wind speed and wind direction": "WIND",
    "ActivePower": "PWR",
    "Air density": "AIR"
}

In [28]:
# Se procesan los 4 csvs por dia y los une
def procesar_dia(day_folder):
    dfs = []
    archivos_faltantes = []

    for keyword, prefix in FILES_CONFIG.items():
        try:
            #case-insensitive para manejar cambios en nombres
            file_path = None
            for archivo in day_folder.glob("*.csv"):
                if keyword.lower() in archivo.name.lower():
                    file_path = archivo
                    break
            
            if file_path is None:
                raise StopIteration
            
            df = pd.read_csv(file_path)
            df.columns = [c.strip() for c in df.columns]

            if 'Time' not in df.columns:
                print(f"'Time' no aparece en {file_path.name}")
                continue
            
            df["Time"] = pd.to_datetime(df["Time"], errors='coerce')
            df = df.dropna(subset=['Time'])
            
            if df.empty:
                print(f"{file_path.name} no tiene datos válidos")
                continue

            cols_data = [c for c in df.columns if c not in ["Time", "Number", "Wind turbine"]]
            rename_map = {c: f"{prefix}_{c}" for c in cols_data}
            df = df.rename(columns=rename_map)
            df = df[["Time"] + list(rename_map.values())]
            df = df.groupby("Time").mean().reset_index()
            df = df.set_index("Time")
            dfs.append(df)
        except StopIteration:
            archivos_faltantes.append(keyword)
        except Exception as e:
            print(f"Error leyendo {keyword} en {day_folder.name}: {e}")
    
    if archivos_faltantes:
        print(f"Faltan: {', '.join(archivos_faltantes)}")
    
    if dfs:
        full_df = pd.concat(dfs, axis=1, join='outer')
        return full_df.reset_index()
    return None

In [29]:
def main():
    total_dias = 0

    # procesar los dos años
    for ROOT_PATH in ROOT_PATHS:
        if not ROOT_PATH.exists():
            print(f"No existe: {ROOT_PATH}")
            continue
        
        print(f"\nProcesando año: {ROOT_PATH}")
        
        # carpetas de meses
        for month_folder in sorted(ROOT_PATH.glob("*_*")):
            if not month_folder.is_dir():
                continue
            
            print(f"\nMes: {month_folder.name}")
            
            # carpetas de dias
            for day_folder in sorted(month_folder.glob("202*")):
                if not day_folder.is_dir():
                    continue
                
                # Verificar si ya existe el archivo procesado
                output_file = OUTPUT_PATH / f"data_{day_folder.name}.parquet"
                if output_file.exists():
                    continue

                print(f"Procesando: {day_folder.name}")
                df_dia = procesar_dia(day_folder)

                if df_dia is not None and not df_dia.empty:
                    df_dia.to_parquet(output_file, index=False)
                    total_dias += 1
                    print(f"Guardado: {output_file.name}")
                else:
                    print(f"No se generó data para {day_folder.name}")

    print(f"Total de dias procesados: {total_dias}")

if __name__ == "__main__":
    main()


Procesando año: 0 REPORTES DIARIOS 2024

Mes: 01_Enero
Procesando: 20240101
Guardado: data_20240101.parquet
Procesando: 20240102
Guardado: data_20240102.parquet
Procesando: 20240103
Guardado: data_20240103.parquet
Procesando: 20240104
Guardado: data_20240104.parquet
Procesando: 20240105
Guardado: data_20240105.parquet
Procesando: 20240106
Guardado: data_20240106.parquet
Procesando: 20240107
Guardado: data_20240107.parquet
Procesando: 20240108
Guardado: data_20240108.parquet
Procesando: 20240109
Guardado: data_20240109.parquet
Procesando: 20240110
Guardado: data_20240110.parquet
Procesando: 20240111
Guardado: data_20240111.parquet
Procesando: 20240112
Guardado: data_20240112.parquet
Procesando: 20240113
Guardado: data_20240113.parquet
Procesando: 20240114
Guardado: data_20240114.parquet
Procesando: 20240115
Guardado: data_20240115.parquet
Procesando: 20240116
Guardado: data_20240116.parquet
Procesando: 20240117
Guardado: data_20240117.parquet
Procesando: 20240118
Guardado: data_2024011

In [30]:
# verificar
parquet_files = sorted(glob.glob("procesados/*.parquet"))
print(f"Total de archivos procesados: {len(parquet_files)}")

if parquet_files:
    sample_file = parquet_files[0]
    df_sample = pd.read_parquet(sample_file)
    
    print(f"\nMuestra de: {sample_file}")
    print(f"   - Shape: {df_sample.shape}")
    print(f"   - Columnas: {df_sample.columns.tolist()}")
    print(f"\n   Primeras filas:")
    print(df_sample.head(3))

Total de archivos procesados: 705

Muestra de: procesados/data_20240101.parquet
   - Shape: (144, 34)
   - Columnas: ['Time', 'GEN_Generator speed-Max', 'GEN_Generator speed-Aver', 'GEN_Generator speed-Min', 'WIND_Wind direction 1s-Max', 'WIND_Wind direction 1s-Aver', 'WIND_Wind direction 1s-Min', 'WIND_Wind direction 10s-Max', 'WIND_Wind direction 10s-Aver', 'WIND_Wind direction 10s-Min', 'WIND_Wind direction 30s-Max', 'WIND_Wind direction 30s-Aver', 'WIND_Wind direction 30s-Min', 'WIND_Wind speed 1s-Max', 'WIND_Wind speed 1s-Aver', 'WIND_Wind speed 1s-Min', 'WIND_Wind speed 30s-Max', 'WIND_Wind speed 30s-Aver', 'WIND_Wind speed 30s-Min', 'WIND_Wind speed 10min-Max', 'WIND_Wind speed 10min-Aver', 'WIND_Wind speed 10min-Min', 'PWR_TotalActivePower-Max', 'PWR_TotalActivePower-Aver', 'PWR_TotalActivePower-Min', 'PWR_TotalRevActivePower-Max', 'PWR_TotalRevActivePower-Aver', 'PWR_TotalRevActivePower-Min', 'PWR_TotalReactivePower-Max', 'PWR_TotalReactivePower-Aver', 'PWR_TotalReactivePower-